In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [2]:
system_prompt = "You are an assistant that analyzes the contents of a website \
                and provides a short summary, ignoring text that might be navigation related. \
                Respond in markdown."

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

def summarize(url):
    website = Website(url) 
    response = ollama_via_openai.chat.completions.create(
    model="llama3.2",
    messages=messages_for(website)
    )
    return response.choices[0].message.content

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [3]:
display_summary("https://cnn.com")

This is the home page of CNN.com, a 24-hour cable news network that provides up-to-date news, information, and analysis on current events from around the world. The page features various sections, including:

1. **Headlines**: A scrolling list of top news stories, updated in real-time.
2. **Special Reports**: In-depth coverage of major stories, such as politics, business, sports, and more.
3. **Top Stories**: Quick summaries of the day's most important stories.
4. **More News**: Articles on various topics, including crime and justice, politics, business, and entertainment.
5. **Photos**: A collection of photos from around the world, with captions and links to related news stories.
6. **Videos**: A selection of videos, including clips of CNN personalities, live coverage of events, and in-depth interviews.
7. **Buzz**: A section highlighting trending topics on social media and online discussions.
8. **Weighing In**: Op-Ed columns and articles from prominent writers on various issues.
9. **On This Day**: Trivia and interesting facts about historical events and dates.
10. **Features**: Articles on lifestyle, travel, food, technology, and other topics of interest.
11. **CNN 5 Things**: A daily quiz featuring five random questions, updated every day at 6:00 pm ET.
12. **Games**: Crosswords, word games, and other interactive activities for entertainment and challenge.
13. **Transcripts**: Transcribed versions of CNN programs and interviews.

In addition to these sections, the home page also includes:

* A navigation bar with links to different CNN brands, including HLN (Women's Interest Network), CNNMoney, and CNN Video.
* Links to popular CNN podcasts, such as "Anderson Cooper 360°" and "The Axe Files."
* A search bar for quickly finding specific news stories or articles.

This page offers a comprehensive overview of the latest news and developments from around the world, making it an excellent starting point for anyone looking to stay informed about current events.